In [1]:
from Preprocessing import Preprocessing
from utils import Behaviour, Baseline
import Experiments
import matplotlib
matplotlib.use('Agg') # Prevent drawings in this notebook
import matplotlib.pyplot as plt

In [2]:
bids_root = "../data/"
preprocessing = Preprocessing(bids_root, "cache")
data = preprocessing.load_all_subjects()

In [ ]:
#combinations = [(3, Behaviour.SHOOT_BUTTON), (8, Behaviour.MISSILE_HIT_ENEMY)]
combinations = [(i, behaviour) for i in range(65) for behaviour in Behaviour]
peak_to_troughs = Experiments.run(data, combinations, 500, 1, 8)

python(49267) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
  3%|▎         | 13/455 [00:48<25:15,  3.43s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x104ecda20>>
Traceback (most recent call last):
  File "/Users/philipp/miniconda3/envs/eeg/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  3%|▎         | 14/455 [00:51<24:04,  3.27s/it]

In [ ]:
def plot_peak_to_trough_differences(behaviour, baseline, mode):
    data = []
    for electrode_index, peak_to_trough_value in peak_to_troughs[behaviour].items():
        data.append((electrode_index, peak_to_trough_value[mode][baseline]))
    title = f"Peak-to-Trough Differences for {behaviour.value} ({"Raw" if mode == 0 else "ICA"}, {baseline.value})"
    sorted_values = sorted(data, key=lambda x:x[1])[-20:]
    labels, values = zip(*sorted_values)
    plt.figure()
    plt.barh(range(len(values)), values, color='skyblue')
    plt.xlabel("Peak-to-Trough Difference")
    plt.ylabel("Electrode")
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.savefig(f"../Results/{title}.png")
    plt.close()    

In [ ]:
for behavior in peak_to_troughs:
    for baseline in Baseline:
        plot_peak_to_trough_differences(behavior, baseline, 0)
        plot_peak_to_trough_differences(behavior, baseline, 1)